In [54]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import GradientBoostingRegressor

PATH_TO_TRAIN = 'data/train.csv'
PATH_TO_TEST = 'data/test.csv'


In [55]:

X_train = pd.read_csv(PATH_TO_TRAIN)
y_train = X_train['Price']

X_train.drop('Price',axis=1, inplace=True)

X_test = pd.read_csv(PATH_TO_TEST)

In [56]:
def add_columns(dataframe, column_name):
    df = pd.get_dummies(dataframe[column_name])
    for name in df.columns:
        df.rename(columns={name : column_name + '_' + name}, inplace=True)
    dataframe[df.columns] = df
    dataframe.drop([column_name], axis=1, inplace=True)

In [59]:
X_train.drop('Healthcare_1', inplace=True, axis=1)
X_test.drop('Healthcare_1', inplace=True, axis=1)

add_columns(X_train, 'Ecology_2')
add_columns(X_train, 'Ecology_3')
add_columns(X_train, 'Shops_2')

add_columns(X_test, 'Ecology_2')
add_columns(X_test, 'Ecology_3')
add_columns(X_test, 'Shops_2')


In [60]:
X_train.loc[X_train['LifeSquare'].isna(), 'LifeSquare'] = X_train['Square'] - X_train['KitchenSquare']
X_test.loc[X_test['LifeSquare'].isna(),   'LifeSquare'] = X_test['Square']  - X_test['KitchenSquare']

In [61]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),
                              columns=X_train.columns,
                              index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test),
                             columns=X_test.columns,
                             index=X_test.index)

In [66]:
X_train.shape, X_test.shape

((10000, 21), (5000, 21))

In [63]:
# forest_best = forest_grid.best_estimator_
gbr = GradientBoostingRegressor(max_depth=13, random_state=42)
gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

array([155813.03064684, 244354.20385332, 188510.1864873 , ...,
       344886.42979929, 224493.50977966, 166268.50417804])

In [67]:
preds_final = pd.DataFrame()
preds_final['Id'] = X_test['Id'].copy()

X_test.set_index('Id', inplace=True)

submission_df = pd.read_csv('data/sample_submission.csv')

submission_df['Price'] = y_pred
submission_df.to_csv('./predictions.csv', index=False, encoding='utf-8', sep=',')

submission_df.head()

,Id,Price
0,725,155813.030647
1,15856,244354.203853
2,5480,188510.186487
3,15664,352600.352872
4,14275,150261.328813
